In [2]:
import os 
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown,display,update_display

In [ ]:
load_dotenv() # we're saving openai api key as env. variable here 
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API Key found !")
elif not api_key.startswith("sk-proj-"):
    print("Not a OpenAI API key")
elif api_key.strip() != api_key:
    print("Remove blank spaces From API key")
else:
    print('API key is found and loaded')

In [4]:
openai = OpenAI()

In [4]:
def call_gpt(user_msg):

    system_msg = "be a helpful AI_assisant"
    

    stream = openai.chat.completions.create(
        
        model='gpt-4o-mini',

        messages=[
            {'role':'system','content':system_msg},
            {'role':'user','content':user_msg}
        ],

        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [ ]:
# THis is a basic structure here we're not sliding a context window as we progressively ask more questions 

user_msg = ""
temp_msg = "temp"

while temp_msg:
    temp_msg = input("Enter User Input : ")
    user_msg += temp_msg
    print("User Message : "+temp_msg)
    print("GPT Response : ")
    gpt_response = call_gpt(user_msg)
    user_msg += str(gpt_response)

### **Convo-History Ollama**

In [7]:
import ollama
from IPython.display import Markdown,display,update_display

def chat_ollama(message, history):

    formatted_history = []

    for user_message, assistant_message in history:
        formatted_history.append({'role': 'user', 'content': user_message})
        formatted_history.append({'role': 'assistant', 'content': assistant_message})

    stream = ollama.chat(
        model='llama3.2',
        messages=[
            {'role': 'system', 'content': "You are an assistant who always lies. Answer the current question based solely on the knowledge you have and the context from the conversation history if it is relevant. Do not explicitly mention or reference the history unless explicitly asked to recall past interactions."},
            {'role': 'user', 'content': f"Conversation History: {str(formatted_history)}\nQuestion: {message}"},
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""),display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(response,display_id=display_handle.display_id)

def main():
    history = []
    user = ''
    
    while user != 'exit': 
        user = input("Enter Question : ") 
        print("User Message :",user)
        print("Model response : ") 
        response = chat_ollama(user,history)
        history.append((user,response))


In [ ]:
main()

### **Conversation history with Chat-GPT**
 Context Loop Using Specific GPT Structure of User , System , Assistant

In [ ]:
import openai
from IPython.display import display, Markdown

def call_gpt2(messages):
    # Initialize with system message
    conversation = [
        {'role': 'system', 'content': "be a helpful AI_assistant"}
    ]
    
    # Add all previous messages to conversation
    conversation.extend(messages)
    
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',  # Make sure this is the correct model name
        messages=conversation,
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        if chunk.choices[0].delta.content:
            response += chunk.choices[0].delta.content
            response_clean = response.replace("```", "").replace("markdown", "")
            display_handle.update(Markdown(response_clean))
    
    return response

def chat_loop():
    # Store the conversation history as a list of message dictionaries
    conversation_history = []
    
    while True:
        user_msg = input("Enter Message (or 'quit' to exit): ")
        if user_msg.lower() == 'quit':
            break
            
        # Add user message to history
        conversation_history.append({
            'role': 'user',
            'content': user_msg
        })
        
        print("User Message : ",user_msg)
        
        # Get GPT response
        print("GPT Response : ")
        gpt_response = call_gpt2(conversation_history)
        
        # Add GPT response to history
        conversation_history.append({
            'role': 'assistant',
            'content': gpt_response
        })
        
        # Optional: Keep only last N messages for context window
        max_context = 10  # Adjust as needed
        if len(conversation_history) > max_context * 2:  # *2 because each exchange has 2 messages
            conversation_history = conversation_history[-max_context * 2:]

# Start the chat
chat_loop()